### Analysis



In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
# Extrating Raw Data for NY State for 2020 
csv_file_path_2020 = '../NY-Healthcare-Data-Visualization/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2020.csv'
df_2020 = pd.read_csv(csv_file_path_2020)
df_2020.head()

/var/folders/j7/xwjqw5s538sc5lrjg_1p04bm0000gn/T/ipykernel_40811/978401291.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv(csv_file_path_2020)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Moderate,Medical,Medicare,NaN,NaN,NaN,Y,214778.30,40532.42
1,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,N,33385.21,8996.74
2,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,50 to 69,104,F,Other Race,Spanish/Hispanic,...,Moderate,Major,Medical,Medicare,Medicaid,NaN,NaN,Y,16581.52,1735.53
3,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,0 to 17,104,M,Other Race,Spanish/Hispanic,...,Moderate,Moderate,Medical,Medicaid,NaN,NaN,NaN,N,14086.86,3438.63
4,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,0 to 17,104,M,Other Race,Spanish/Hispanic,...,Moderate,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,NaN,Y,28222.99,6886.04


In [3]:
# Extrating Raw Data for NY State for 2021
csv_file_path_2021 = '../NY-Healthcare-Data-Visualization/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021.csv'
df_2021 = pd.read_csv(csv_file_path_2021)
df_2021.head()

/var/folders/j7/xwjqw5s538sc5lrjg_1p04bm0000gn/T/ipykernel_40811/890408147.py:3: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv(csv_file_path_2021)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,70 or Older,104,M,Other Race,Spanish/Hispanic,...,Major,Extreme,Medical,Medicare,Medicaid,NaN,NaN,Y,320922.43,60241.34
1,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,White,Not Span/Hispanic,...,Moderate,Minor,Medical,Private Health Insurance,NaN,NaN,NaN,Y,61665.22,9180.69
2,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,F,Other Race,Spanish/Hispanic,...,Minor,Minor,Surgical,Medicaid,NaN,NaN,NaN,N,42705.34,11366.50
3,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,70 or Older,104,M,Other Race,Spanish/Hispanic,...,Major,Major,Medical,Medicare,Medicaid,NaN,NaN,Y,72700.17,12111.75
4,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicare,Medicaid,NaN,NaN,Y,55562.51,8339.72


In [4]:
# Extrating Raw Data for NY State for 2022 
csv_file_path_2022 = '../NY-Healthcare-Data-Visualization/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2022.csv'
df_2022 = pd.read_csv(csv_file_path_2022)
df_2022.head()

/var/folders/j7/xwjqw5s538sc5lrjg_1p04bm0000gn/T/ipykernel_40811/697135797.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv(csv_file_path_2022)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,107,F,White,Not Span/Hispanic,...,Major,Major,Medical,Medicaid,NaN,NaN,NaN,Y,"51,514.62","7,552.54"
1,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,104,M,Black/African American,Spanish/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,Y,"25,370.86","3,469.55"
2,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,104,F,Other Race,Spanish/Hispanic,...,Minor,Minor,Medical,Medicaid,NaN,NaN,NaN,N,"23,876.78","6,180.33"
3,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,100,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,Y,"43,319.05","12,588.93"
4,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,M,Other Race,Spanish/Hispanic,...,Moderate,Moderate,Medical,Medicaid,NaN,NaN,NaN,Y,"40,266.23","10,355.99"
